In [ ]:
# install required packages
# %pip install torch

In [ ]:
# read the file
data_file = "data/tinyshakespeare.txt"
with open(data_file, "r", encoding="utf-8") as file:
    text = file.read()

print("Length of text: {} characters".format(len(text)))

In [ ]:
# vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("Number of unique characters: {}".format(vocab_size))

In [ ]:
# create a mapping from unique characters to indices
stoi = {c: i for i, c in enumerate(chars)}
itos = {i: c for i, c in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("hello world"))
print(decode(encode("hello world")))

In [ ]:
import torch

# create a tensor from the text
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

# split data into train and valiation sets
tn = int(len(data) * 0.9)
train_data, val_data = data[:tn], data[tn:]

block_size = 8
train_data[: block_size + 1]

x = train_data[:block_size]
y = train_data[1 : block_size + 1]
for i in range(block_size):
    content = x[: i + 1]
    target = y[i]
    print(f"content: {content}, target: {target}")

In [ ]:
torch.manual_seed(1337)
block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == "train" else val_data
    idx = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in idx])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in idx])
    return x, y

xb, yb = get_batch("train")
print("input")
print(xb)
print("target")
print(yb)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        # idx and targets are both (B,T) tensor
        logits = self.token_embedding_table(idx)  # (B,T,C)
        logits = logits.view(-1, self.token_embedding_table.embedding_dim)
        targets = targets.view(-1)
        loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_token):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_token):
            # get the predictions
            logits = self.token_embedding_table(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = torch.softmax(logits, dim=1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

out = m.generate(torch.zeros((1, 1), dtype=torch.long), 50)
print(decode(out[0].tolist()))

In [ ]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
# training loop
batch_size = 32
for _ in range(100):
    # sample batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    optimizer.zero_grad()
    logits, loss = m(xb, yb)
    loss.backward()
    optimizer.step()

    print(loss.item())

In [ ]:
out = m.generate(torch.zeros((1, 1), dtype=torch.long), 300)
print(decode(out[0].tolist()))

### The mathematical trick in self-attention

In [ ]:
import torch

# consider the following example
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

In [ ]:
# we want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xbow[b, t] = x[b, : t + 1].mean(dim=0)

print(x[1])
print(xbow[1])

In [ ]:
# version 2: using matrix multipl
wei = torch.tril(torch.ones((T, T)))
wei = wei / wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x # (B,T,T) @ (B,T,C) -> (B,T,C)

print(torch.allclose(xbow, xbow2, atol=1e-7))

In [ ]:
# version 3: using softmax
mask = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
wei = wei.masked_fill(mask == 0, float('-inf'))
wei = torch.softmax(wei, dim=1)
xbow3 = wei @ x # (B,T,T) @ (B,T,C) -> (B,T,C)

print(torch.allclose(xbow, xbow3, atol=1e-7))